In [1]:
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import open_clip

In [2]:
DEVICE = 'cuda:3' if torch.cuda.is_available() else 'cpu'
print(f'Using {DEVICE}')

Using cuda:3


In [3]:
def conv1d_calc(in_feats, kernel_size, stride=1, padding=0, dilation=1):
    return ((in_feats + 2*padding - dilation * (kernel_size - 1) - 1)/stride) + 1

def conv1dtranspose_calc(in_feats, kernel_size, stride=1, padding=0, output_padding=0, dilation=1):
    return (in_feats - 1) * stride - 2 * padding + dilation*(kernel_size-1) + output_padding + 1

# conv1d_calc(77, 21, padding=10)
# conv1d_calc(77, 15, stride=2)
# conv1d_calc(32, 11)
# conv1d_calc(22, 7)

conv1dtranspose_calc(77, 21)
conv1dtranspose_calc(97, 15, dilation=9)
conv1d_calc(223, 15)
conv1d_calc(209, 15, stride=2)
# conv1d_calc(109, 7)
# conv1dtranspose_calc(97, 21, dilation=2)
# conv1dtranspose_calc(137, 9, dilation=5)
# conv1dtranspose_calc(177, 3, dilation=32, output_padding=15)

98.0

In [4]:
class EncBlock(nn.Module):
    def __init__(self, input_dim= 1024, output_dim=1024):
        super(EncBlock, self).__init__()
        self.fx = nn.Sequential(
            nn.Linear(input_dim, int(input_dim / 2)),
            nn.Tanh(),
            nn.Linear(int(input_dim / 2), int(input_dim / 2)),
            nn.Tanh(),
            nn.Linear(int(input_dim / 2), int(input_dim / 4)),
            nn.Tanh(),
            nn.Linear(int(input_dim / 4), int(input_dim / 2)),
            nn.Tanh(),
            nn.Linear(int(input_dim / 2), int(output_dim))
        )
        
    def forward(self, x):
        return self.fx(x)
    
    
class ResBlock(nn.Module):
    def __init__(self):
        super(ResBlock, self).__init__()
        pass
    
    def forward(self, x, block: callable):
        x = x + block(x)
        return x
        

In [5]:
class BadNet(nn.Module):
    def __init__(self, token_dim=77, output_dim=1024, normalize=False):
        super(BadNet, self).__init__()

        self.pos_enc = nn.Sequential(
            nn.Linear(77, 77),
            nn.Tanh(),
            nn.Linear(77, 77),
            nn.Tanh(),
            nn.Linear(77, 77),
            nn.Sigmoid()
        )
        
        self.encBlocks = nn.ModuleList([EncBlock(input_dim=1024, output_dim=1024) for _ in range(5)])
        self.resBlock = ResBlock()
                
        self.seq = nn.Sequential(
            nn.ConvTranspose1d(in_channels=1, out_channels=4, kernel_size=21),
            nn.Tanh(),
            nn.ConvTranspose1d(in_channels=4, out_channels=4, kernel_size=15, dilation=9),
            nn.Tanh(),
            nn.Conv1d(in_channels=4, out_channels=4, kernel_size=15),
            nn.Tanh(),
            nn.Conv1d(in_channels=4, out_channels=4, kernel_size=15, stride=2),
            nn.Flatten(start_dim=1, end_dim=2),
            nn.Linear(392, 1024),
            nn.Tanh()
        )

        self.normalize = normalize

    def forward(self, x):
        x = x + self.pos_enc(x)
        x = x.unsqueeze(1)
        x = self.seq(x)
        
        for block in self.encBlocks:
            x = self.resBlock(x, block)

        return x

In [6]:
class BabySet(data.Dataset):
    def __init__(self, token_file, feature_file):
        self.features = torch.load(feature_file)
        self.tokens = torch.load(token_file)
        assert(self.features.shape[0] == self.tokens.shape[0])
    
    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self, index):
        return  self.tokens[index], self.features[index]

babyset = BabySet( './data/tokens.pt', './data/features.pt')
print(len(babyset))
print(babyset[0])
print(torch.max(babyset[0][1]))

trainset, validset, testset = data.random_split(babyset, [0.8, 0.1, 0.1])

67780
(tensor([49406.,   320.,  2308.,   530.,   320.,  1530.,   593.,   320.,  2368.,
          269., 49407.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.]), tensor([ 0.0289, -0.0060, -0.0365,  ..., -0.0244, -0.0228, -0.0224]))
tensor(0.2090)


In [7]:
BATCH_SIZE = 1024
NUM_WORKERS = 0

trainloader = data.DataLoader(trainset, shuffle=True, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
validloader = data.DataLoader(validset, shuffle=True, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
testloader = data.DataLoader(testset, shuffle=True, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

In [8]:
def run_epoch(model, dataloader, optimizer, criterion, metric, train, verbose=False):
    if train:
        model.train()
    else:
        model.eval()
    with torch.set_grad_enabled(train):
        t = tqdm(dataloader)
        losses = torch.zeros(len(t))
        accs = torch.zeros(len(t))
        for i, (X, y) in enumerate(t):
            optimizer.zero_grad()
            X, y = X.to(DEVICE), y.to(DEVICE)
            pred = model(X)
            loss = criterion(pred, y)
            if train:
                loss.backward()
                optimizer.step()
            acc = metric(pred, y).mean()
            if verbose:
                t.set_description(f'Loss = {loss:.4f}, Accuracy = {acc * 100:02.2f}%')
            losses[i] = loss.detach().cpu().item()
            accs[i] = acc.detach().cpu().item()
    return losses, accs

In [9]:
model = BadNet(normalize=False).to(DEVICE)

# for p in model.parameters():
#     if p.dim() > 1:
#         nn.init.xavier_uniform_(p)
        
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()
metric = nn.CosineSimilarity()

EPOCHS = 100

train_losses = []
train_accs = []
valid_losses = []
valid_accs = []
for epoch in range(EPOCHS):
    print(f'===== EPOCH {epoch+1:02} =====')
    print('Training...')
    epoch_train_losses, epoch_train_accs = run_epoch(model, trainloader, optimizer, criterion, metric, train=True)
    train_losses.append(epoch_train_losses.mean())
    train_accs.append(epoch_train_accs.mean())
    print(f'Epoch Train Loss = {train_losses[-1]:.4f}, Epoch Train Accuracy = {train_accs[-1] * 100:02.2f}%')

    print('Validating...')
    epoch_valid_losses, epoch_valid_accs = run_epoch(model, validloader, optimizer, criterion, metric, train=False)
    valid_losses.append(epoch_valid_losses.mean())
    valid_accs.append(epoch_valid_accs.mean())
    print(f'Epoch Validation Loss = {valid_losses[-1]:.4f}, Epoch Validation Accuracy = {valid_accs[-1] * 100:02.2f}%')

print(f'===== TESTING =====')
test_losses, test_accs = run_epoch(model, testloader, optimizer, criterion, metric, train=False)
print(f'Test Loss = {test_losses.mean():.4f}, Test Accuracy = {test_accs.mean() * 100:02.2f}%')

===== EPOCH 01 =====
Training...


100%|██████████| 53/53 [00:02<00:00, 20.57it/s]


Epoch Train Loss = 0.0013, Epoch Train Accuracy = 37.18%
Validating...


100%|██████████| 7/7 [00:00<00:00, 124.38it/s]


Epoch Validation Loss = 0.0008, Epoch Validation Accuracy = 48.16%
===== EPOCH 02 =====
Training...


...

Epoch Train Loss = 0.0007, Epoch Train Accuracy = 53.80%
Validating...


100%|██████████| 7/7 [00:00<00:00, 131.91it/s]


Epoch Validation Loss = 0.0007, Epoch Validation Accuracy = 53.79%
===== TESTING =====


100%|██████████| 7/7 [00:00<00:00, 134.29it/s]

Test Loss = 0.0007, Test Accuracy = 53.67%
